In [5]:
import igl
import numpy as np
from gpytoolbox import cotangent_laplacian
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from meshplot import plot, subplot, interact

In [6]:
#V, F = igl.read_triangle_mesh("bar1.off")
V, F = igl.read_triangle_mesh("Meshes_ARAP_SorkineAlexa_2007/Meshes_ARAP_SA2007/cactus_small.off")

print(len(V))
print(len(F))
plot(V,F)

620
1236


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4999655…

In [7]:
L = cotangent_laplacian(V, F)

In [8]:
fixed_vertex_index = 0
fixed_vertex_position = V[fixed_vertex_index, :]
print(fixed_vertex_position)

target_vertex_index =5
target_position = np.array([2.0, 1.0, 0.0])

print(target_position)

[0.51321697 0.45633    0.582147  ]
[2. 1. 0.]


In [9]:
# Boundary conditions : Create a list of indices to fix & matrix of target positions for those vertices
boundary_conditions = {fixed_vertex_index: fixed_vertex_position,
                       target_vertex_index: target_position}

fixed_indices = np.array(list(boundary_conditions.keys()))
fixed_positions = np.array([boundary_conditions[i] for i in fixed_indices])
print(fixed_indices)
print(fixed_positions)

[0 5]
[[0.51321697 0.45633    0.582147  ]
 [2.         1.         0.        ]]


In [10]:
#Initialize a default deformed vertices
deformedV = V.copy()

In [11]:
max_iterations = 10

for iter in range(max_iterations):
    # Step 1: Compute local rotations
    for i in range(V.shape[0]):
        # Compute the neighborhood matrix S
        # For simplicity, let's assume we are using the Laplacian's neighbors
        neighbors = F[np.any(F == i, axis=1)]
        S = deformedV[neighbors[:, 1:], :] - deformedV[i, :]

        # Check if S is rank deficient
        if S.shape[0] < 3 or S.shape[1] < 3:
            continue  # Skip this vertex if S is too small
            
        # SVD decomposition of S
        U, D, Vh = np.linalg.svd(S)
        
        # Ensure that U and Vh have the correct dimensions (3x3)
        if U.shape[1] == 3 and Vh.shape[0] == 3:
            R = U @ Vh
        else:
            R = np.eye(3)  # In case of degenerate cases, set R to identity
        
        #R = U @ Vh.T
        
        # Update the position with rotation (this is simplified, normally we'd calculate energy minimization)
        deformedV[i, :] = np.dot(R, V[i, :].T).T
    
    # Step 2: Solve the linear system to find the new positions
    RHS = -L @ deformedV
    RHS[fixed_indices, :] = fixed_positions
    L[fixed_indices, :] = 0
    L[fixed_indices, fixed_indices] = 1

    # Solve the system
    deformedV = spsolve(csr_matrix(L), RHS)

D:\Anaconda\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [12]:
plot(deformedV,F)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.2566084…

In [ ]:
# Save the ARAP deformed mesh
igl.write_triangle_mesh("bar1_arap_deformed.off", deformedV, F)